<a href="https://colab.research.google.com/github/segzy01/Oluwasegun-Durowoju-Prework/blob/master/AI_Slide_SeqTo1_VowelSounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount gdrive with this code
from google.colab import drive
drive.mount('/content/drive')
#below where the file is in gdrive, change with yours
data_path = "/content/drive/My Drive/AI_764/"

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
sheetData = pd.read_excel('/content/drive/My Drive/AI_764/VowelSound_Padded.xlsx', 
                          sheet_name='XTrain', header=None)
XTrain = np.array(sheetData)
print(XTrain.shape, XTrain.size)

sheetData = pd.read_excel('/content/drive/My Drive/AI_764/VowelSound_Padded.xlsx', 
                          sheet_name='XTest', header=None)
XTest = np.array(sheetData)
print(XTest.shape, XTest.size)


sheetData = pd.read_excel('/content/drive/My Drive/AI_764/VowelSound_Padded.xlsx', 
                          sheet_name='YTrain', header=None)
YTrain = np.array(sheetData)
YTrain = YTrain - 1   # from 0 to 8
print(YTrain.shape, YTrain.size)

sheetData = pd.read_excel('/content/drive/My Drive/AI_764/VowelSound_Padded.xlsx', 
                          sheet_name='YTest', header=None)
YTest = np.array(sheetData)
YTest = YTest - 1   # from 0 to 8
print(YTest.shape, YTest.size)

(3240, 29) 93960
(4440, 29) 128760
(270, 1) 270
(370, 1) 370


In [ ]:
NumVars = 12
numClasses = 9
SeqLength = 29    # all variable length sequences have been preprocessed and padded to longest 29
NumTrainRecs = int(XTrain.shape[0] / NumVars)
NumTestRecs = int(XTest.shape[0] / NumVars)
print(NumTrainRecs, NumTestRecs)

270 370


In [ ]:
# make sure the size being read in is correct
XTrain = XTrain.reshape(NumTrainRecs, SeqLength, NumVars)
print(XTrain.shape, XTrain.size)

XTest = XTest.reshape(NumTestRecs, SeqLength, NumVars)
print(XTest.shape, XTest.size)

(270, 29, 12) 93960
(370, 29, 12) 128760


In [ ]:
# Convert array of labeled data to one-hot vector
from keras.utils.np_utils import to_categorical

CAT_YTrain = to_categorical(YTrain)
print(np.min(YTrain), np.max(YTrain), CAT_YTrain.shape)

CAT_YTest = to_categorical(YTest)
print(np.min(YTest), np.max(YTest), CAT_YTest.shape)

0 8 (270, 9)
0 8 (370, 9)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM

In [ ]:
numHiddenUnits = 100
maxEpochs = 100;
miniBatchSize = 27;

model = Sequential()
model.add(LSTM(numHiddenUnits, input_shape=(SeqLength, NumVars)))
model.add(Dense(numClasses, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               45200     
_________________________________________________________________
dense (Dense)                (None, 9)                 909       
Total params: 46,109
Trainable params: 46,109
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(XTrain, CAT_YTrain, epochs = maxEpochs, batch_size=  miniBatchSize, verbose = 1)

Epoch 1/100
10/10 [==============================] - 5s 5ms/step - loss: 2.1954 - accuracy: 0.1444
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 2.1758 - accuracy: 0.2037
Epoch 3/100
10/10 [==============================] - 0s 5ms/step - loss: 2.0584 - accuracy: 0.2704
Epoch 4/100
10/10 [==============================] - 0s 5ms/step - loss: 1.8209 - accuracy: 0.3148
Epoch 5/100
10/10 [==============================] - 0s 5ms/step - loss: 1.6706 - accuracy: 0.3593
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 1.5773 - accuracy: 0.3519
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 1.4739 - accuracy: 0.3889
Epoch 8/100
10/10 [==============================] - 0s 4ms/step - loss: 1.4009 - accuracy: 0.4407
Epoch 9/100
10/10 [==============================] - 0s 4ms/step - loss: 1.3284 - accuracy: 0.4778
Epoch 10/100
10/10 [==============================] - 0s 4ms/step - loss: 1.3759 - accuracy: 0.5000
Epoch 11/

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# evaluate predictions from training
Y_score_train = model.predict(XTrain, verbose=0)
Y_hat_train = np.argmax(Y_score_train, axis=-1)   
print("Classification Report: \n \n", classification_report(YTrain, Y_hat_train))
print("Confusion matrix: \n \n", confusion_matrix(YTrain, Y_hat_train))

Classification Report: 
 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       1.00      1.00      1.00        30
           2       1.00      0.87      0.93        30
           3       0.91      1.00      0.95        30
           4       1.00      0.93      0.97        30
           5       1.00      1.00      1.00        30
           6       0.94      1.00      0.97        30
           7       0.81      1.00      0.90        30
           8       1.00      0.80      0.89        30

    accuracy                           0.96       270
   macro avg       0.96      0.96      0.96       270
weighted avg       0.96      0.96      0.96       270

Confusion matrix: 
 
 [[30  0  0  0  0  0  0  0  0]
 [ 0 30  0  0  0  0  0  0  0]
 [ 0  0 26  3  0  0  0  1  0]
 [ 0  0  0 30  0  0  0  0  0]
 [ 0  0  0  0 28  0  0  2  0]
 [ 0  0  0  0  0 30  0  0  0]
 [ 0  0  0  0  0  0 30  0  0]
 [ 0  0  0  0  0  0  0 30  0]
 [ 0  

In [ ]:
# evaluate predictions from test
Y_score_test = model.predict(XTest, verbose=0)
Y_hat_test = np.argmax(Y_score_test, axis=-1)   
print("Classification Report: \n \n", classification_report(YTest, Y_hat_test))
print("Confusion matrix: \n \n", confusion_matrix(YTest, Y_hat_test))

Classification Report: 
 
               precision    recall  f1-score   support

           0       0.96      0.81      0.88        31
           1       0.85      0.83      0.84        35
           2       0.90      0.81      0.85        88
           3       0.77      0.91      0.83        44
           4       1.00      0.90      0.95        29
           5       0.88      0.92      0.90        24
           6       0.90      0.68      0.77        40
           7       0.71      0.90      0.80        50
           8       0.66      0.79      0.72        29

    accuracy                           0.83       370
   macro avg       0.85      0.84      0.84       370
weighted avg       0.85      0.83      0.83       370

Confusion matrix: 
 
 [[25  0  0  0  0  3  0  1  2]
 [ 0 29  0  4  0  0  0  2  0]
 [ 0  2 71  8  0  0  0  6  1]
 [ 0  1  3 40  0  0  0  0  0]
 [ 0  0  0  0 26  0  0  3  0]
 [ 0  1  0  0  0 22  0  1  0]
 [ 0  0  1  0  0  0 27  3  9]
 [ 0  1  4  0  0  0  0 45  0]
 [ 1  